## RAG with Strands Agents SDK

RAG architectures have proven effective at leveraging knowledge bases to enhance foundation model outputs. However, for more complex queries that require reasoning over diverse information sources, a single monolithic RAG model can face limitations around relevance, latency, and coherence. Multi-agent architectures offer a powerful way to overcome these limitations by factoring RAG into specialized components.

In this lab, we are going to extend the Technical document assistant from previous Naive RAG lab and build an agent using **Strands Agents SDK** that can generate API flow diagrams, create unit testing code, and retrieve knowledge from a knowledge base. The Strands SDK provides a much simpler, more Pythonic way to build agents compared to the traditional boto3 Bedrock Agent API.

**Key benefits of Strands SDK:**
- Simple agent creation with `Agent(tools=[...])`
- Tools defined as Python functions with `@tool` decorator
- Direct invocation with `agent(query)`
- No complex setup with action groups, aliases, or Lambda permissions
- Full observability and tracing support

![Agent](../static/advance-agent-rag.png)

## Pre-req
You must run the [workshop_setup.ipynb](../lab00-setup/workshop_setup.ipynb) notebook in `lab00-setup` before starting this lab.

In [ ]:
import warnings
warnings.warn("Warning: if you did not run lab00-setup, please go back and run the lab00 notebook")

## Install Strands Agents SDK

First, we need to install the Strands Agents SDK and related dependencies.

In [ ]:
!pip install -q strands-agents strands-agents-tools

## Load the parameters

In [ ]:
print("load the data parameters....\n")
# bucket and parameter stored from Initial setup lab01
%store -r bucket
%store -r prefix
%store -r data_dir
%store -r yml_dir
%store -r uml_dir

## check all 5 values are printed and do not fail
print(bucket)
print(prefix)
print(yml_dir)
print(uml_dir)
print(data_dir)

print("\nload the vector db parameters....\n")

# vector parameters stored from Initial setup lab02
%store -r vector_host
%store -r vector_collection_arn
%store -r vector_collection_id
%store -r bedrock_kb_execution_role_arn

print(vector_host)
print(vector_collection_arn)
print(vector_collection_id)
print(bedrock_kb_execution_role_arn)

## Setup

In [ ]:
import boto3
from botocore.config import Config
import time
import random
import pprint as pp
import uuid
import json
from retrying import retry
from utility import create_bedrock_execution_role, create_oss_policy_attach_bedrock_execution_role, create_policies_in_oss
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# auth for opensearch
boto3_config = Config(
        connect_timeout=1, read_timeout=300,
        retries={'max_attempts': 1})

boto3_session = boto3.Session()
region_name = boto3_session.region_name
sts_client = boto3.client('sts')
account_id = sts_client.get_caller_identity()["Account"]
credentials = boto3_session.get_credentials()

# opensearch service
service = 'aoss'
awsauth = auth = AWSV4SignerAuth(credentials, region_name, service)

suffix = random.randrange(200, 900)

bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=region_name)

## Create a vector store - OpenSearch Serverless index

For this lab, we will use *Amazon OpenSearch serverless.*

Amazon OpenSearch Serverless is a serverless option in Amazon OpenSearch Service. As a developer, you can use OpenSearch Serverless to run petabyte-scale workloads without configuring, managing, and scaling OpenSearch clusters. You get the same interactive millisecond response times as OpenSearch Service with the simplicity of a serverless environment. Pay only for what you use by automatically scaling resources to provide the right amount of capacity for your application—without impacting data ingestion.

In [ ]:
aoss_client = boto3_session.client('opensearchserverless')

### Create the schema for vector index

In [ ]:
index_name = f"bedrock-sample-index-{suffix}"
body_json = {
   "settings": {
      "index.knn": "true"
   },
   "mappings": {
      "properties": {
         "vector": {
            "type": "knn_vector",
            "dimension": 1024,
            "method": {
                "name": "hnsw",
                "space_type": "innerproduct",
                "engine": "faiss",
                "parameters": {
                  "ef_construction": 256,
                  "m": 48
                }
             }
         },
         "text": {
            "type": "text"
         },
         "text-metadata": {
            "type": "text"         
         }
      }
   }
}
# Build the OpenSearch client
oss_client = OpenSearch(
    hosts=[{'host': vector_host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    timeout=300
)

In [ ]:
# Create index
response = oss_client.indices.create(index=index_name, body=json.dumps(body_json))
print('\nCreating index:')
print(response)

## Create Knowledge Base
Steps:
- initialize OpenSearch serverless configuration which will include collection ARN, index name, vector field, text field and metadata field.
- initialize chunking strategy, based on which KB will split the documents into pieces of size equal to the chunk size mentioned in the `chunkingStrategyConfiguration`.
- initialize the s3 configuration, which will be used to create the data source object later.
- initialize the Titan embeddings model ARN, as this will be used to create the embeddings for each of the text chunks.

In [ ]:
opensearchServerlessConfiguration = {
            "collectionArn": vector_collection_arn,
            "vectorIndexName": index_name,
            "fieldMapping": {
                "vectorField": "vector",
                "textField": "text",
                "metadataField": "text-metadata"
            }
        }

chunkingStrategyConfiguration = {
    "chunkingStrategy": "NONE",
}

s3Configuration = {
    "bucketArn": f"arn:aws:s3:::{bucket}",
    "inclusionPrefixes":[f"{prefix}/{yml_dir.replace(data_dir+'/', '')}/"] # you can use this if you want to create a KB using data within s3 prefixes.
}

embeddingModelArn = f"arn:aws:bedrock:{region_name}::foundation-model/amazon.titan-embed-text-v2:0"

kb_name = f"bedrock-sample-knowledge-base-{suffix}"
description = "Swagger OpenAPI knowledge base."

Provide the above configurations as input to the `create_knowledge_base` method, which will create the Knowledge base.

In [ ]:
# Create a KnowledgeBase
from retrying import retry

@retry(wait_random_min=1000, wait_random_max=2000,stop_max_attempt_number=7)
def create_knowledge_base_func():
    create_kb_response = bedrock_agent_client.create_knowledge_base(
        name = kb_name,
        description = description,
        roleArn = bedrock_kb_execution_role_arn,
        knowledgeBaseConfiguration = {
            "type": "VECTOR",
            "vectorKnowledgeBaseConfiguration": {
                "embeddingModelArn": embeddingModelArn
            }
        },
        storageConfiguration = {
            "type": "OPENSEARCH_SERVERLESS",
            "opensearchServerlessConfiguration":opensearchServerlessConfiguration
        }
    )
    return create_kb_response["knowledgeBase"]

In [ ]:
try:
    kb = create_knowledge_base_func()
except Exception as err:
    print(f"{err=}, {type(err)=}")

Next we need to create a data source, which will be associated with the knowledge base created above. Once the data source is ready, we can then start to ingest the documents.

In [ ]:
# Get KnowledgeBase 
get_kb_response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId = kb['knowledgeBaseId'])

In [ ]:
# Create a DataSource in KnowledgeBase 
create_ds_response = bedrock_agent_client.create_data_source(
    name = kb_name,
    description = description,
    knowledgeBaseId = kb['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration":s3Configuration
    },
    vectorIngestionConfiguration = {
        "chunkingConfiguration": chunkingStrategyConfiguration
    }
)
ds = create_ds_response["dataSource"]
# # It can take up to a minute for data access rules to be enforced
time.sleep(20)
pp.pprint(ds)

### Start ingestion job
Once the KB and data source is created, we can start the ingestion job.
During the ingestion job, KB will fetch the documents in the data source, pre-process it to extract text, chunk it based on the chunking size provided, create embeddings of each chunk and then write it to the vector database, in this case OSS.

In [ ]:
# Start an ingestion job
start_job_response = bedrock_agent_client.start_ingestion_job(knowledgeBaseId = kb['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [ ]:
job = start_job_response["ingestionJob"]
pp.pprint(job)

In [ ]:
# Get job 
while(job['status']!='COMPLETE' ):
  get_job_response = bedrock_agent_client.get_ingestion_job(
      knowledgeBaseId = kb['knowledgeBaseId'],
        dataSourceId = ds["dataSourceId"],
        ingestionJobId = job["ingestionJobId"]
  )
  job = get_job_response["ingestionJob"]
pp.pprint(job)
time.sleep(80)

In [ ]:
kb_id = kb["knowledgeBaseId"]
%store kb_id
pp.pprint(kb_id)

## Test the knowledge base
### Using RetrieveAndGenerate API
Before creating our Strands agent, let's test the knowledge base to ensure it's working correctly.

In [ ]:
# try out KB using RetrieveAndGenerate API
bedrock_agent_runtime_client = boto3_session.client("bedrock-agent-runtime", 
                                                    config=boto3_config)
model_id = "anthropic.claude-3-sonnet-20240229-v1:0" 
model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id}'

In [ ]:
from IPython.display import Markdown, display

query = "What APIs are available in the petstore?"
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        'text': query
    },
    retrieveAndGenerateConfiguration={
        'type': 'KNOWLEDGE_BASE',
        'knowledgeBaseConfiguration': {
            'knowledgeBaseId': kb_id,
            'modelArn': model_arn
        }
    },
)

generated_text = response['output']['text']

display(Markdown(generated_text))

## Create the Agent using Strands SDK

Now we'll create our agent using the Strands SDK. This is much simpler than the traditional boto3 approach:

**Traditional Approach (boto3):**
- Create agent via `bedrock_agent_client.create_agent()`
- Create action groups via `create_agent_action_group()`
- Deploy Lambda functions
- Add Lambda permissions
- Create agent alias
- Prepare agent
- Invoke with `invoke_agent()` and parse event stream

**Strands Approach:**
- Define tools with `@tool` decorator
- Create agent: `agent = Agent(tools=[...])`
- Invoke: `agent(query)`

That's it!

In [ ]:
from strands import Agent
from strands_agent_tools import get_uml_diagram, get_unit_test_code, search_knowledge_base

In [ ]:
# Create wrapper tools that include the kb_id
from strands import tool

@tool
def search_swagger_docs(query: str) -> str:
    """
    Search the Swagger API documentation knowledge base to find relevant API information.
    Use this tool to retrieve OpenAPI YAML specifications before generating diagrams or code.
    
    Args:
        query: Search query to find relevant API documentation
    
    Returns:
        JSON string containing the retrieved OpenAPI YAML content and metadata
    """
    return search_knowledge_base(query, kb_id, max_results=5)

# Create the Strands agent with all tools
agent = Agent(
    tools=[search_swagger_docs, get_uml_diagram, get_unit_test_code],
    model="bedrock:anthropic.claude-3-sonnet-20240229-v1:0",
    instructions="""
    You are a helpful assistant for Swagger API developers. You can:
    
    1. Search the Swagger API documentation knowledge base to answer questions
    2. Generate UML flow diagrams from OpenAPI specifications
    3. Generate functional test code in various programming languages
    
    When a user asks about an API:
    - First, search the knowledge base to retrieve the relevant OpenAPI YAML specification
    - Then, use the appropriate tool to generate diagrams or code as requested
    - Always provide clear, helpful responses based on the documentation
    
    If information is not available in the documentation, politely inform the user.
    """
)

print("Strands Agent created successfully!")
print(f"Agent has {len(agent.tools)} tools available")

## Test the Strands Agent

Now let's test our agent with various queries. Notice how simple the invocation is compared to the traditional approach!

### Test 1: Knowledge Base Search
Ask a question about the API documentation

In [ ]:
%%time
query = "How do I add a new pet to the petstore API?"
response = agent(query)
display(Markdown(response))

### Test 2: Generate UML Diagram
Generate a UML flow diagram for an API

In [ ]:
%%time
query = "Can you generate a UML diagram for the petstore API?"
response = agent(query)
display(Markdown(response))

### Test 3: Generate Test Code
Generate Python test code for an API endpoint

In [ ]:
%%time
query = "Generate Python test code to add a new pet to the petstore API"
response = agent(query)
display(Markdown(response))

### Test 4: Multi-turn Conversation
The agent maintains conversation context automatically

In [ ]:
# First turn
query = "What endpoints are available in the bookstore API?"
response = agent(query)
display(Markdown(response))

In [ ]:
# Follow-up question (agent remembers context)
query = "Can you show me how to use the first one in Python?"
response = agent(query)
display(Markdown(response))

## Comparison: Strands SDK vs Traditional Boto3 Approach

### Code Comparison

**Traditional Boto3 Approach (from original notebook):**
```python
# 1. Create agent
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=bedrock_kb_execution_role_arn,
    foundationModel=model_id,
    instruction=agent_instruction,
)
agent_id = response['agent']['agentId']

# 2. Create action group with Lambda
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={'lambda': lambda_arn},
    functionSchema={'functions': agent_functions}
)

# 3. Add Lambda permissions
lambda_client.add_permission(
    FunctionName=lambda_function_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region_name}:{account_id}:agent/{agent_id}",
)

# 4. Associate knowledge base
bedrock_agent_client.associate_agent_knowledge_base(
    agentId=agent_id,
    agentVersion='DRAFT',
    knowledgeBaseId=kb_id,
    knowledgeBaseState='ENABLED'
)

# 5. Prepare agent
bedrock_agent_client.prepare_agent(agentId=agent_id)
time.sleep(30)

# 6. Create alias
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)
time.sleep(60)

# 7. Invoke agent (complex event stream parsing)
def invokeAgent(query, session_id, enable_trace=False):
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=query,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace
    )
    event_stream = agentResponse['completion']
    for event in event_stream:
        if 'chunk' in event:
            return event['chunk']['bytes'].decode('utf8')

response = invokeAgent(query, session_id)
```

**Strands SDK Approach (this notebook):**
```python
# 1. Define tools with @tool decorator (in strands_agent_tools.py)
@tool
def get_uml_diagram(yml_body: str) -> str:
    """Generate UML diagram from OpenAPI spec"""
    # Implementation

# 2. Create agent
agent = Agent(
    tools=[search_swagger_docs, get_uml_diagram, get_unit_test_code],
    model="bedrock:anthropic.claude-3-sonnet-20240229-v1:0",
    instructions="..."
)

# 3. Invoke agent
response = agent(query)
```

### Key Benefits of Strands SDK:

1. **Simplicity**: ~200 lines of setup code reduced to ~20 lines
2. **No Infrastructure**: No Lambda functions, IAM roles, or permissions to manage
3. **Pythonic**: Tools are just Python functions with decorators
4. **Easy Testing**: Test tools directly as Python functions
5. **Quick Iteration**: Change tools and test immediately, no deployment needed
6. **Built-in Features**: Automatic conversation memory, streaming, tracing
7. **Framework Agnostic**: Works with any model provider (Bedrock, OpenAI, etc.)

### When to Use Each Approach:

**Use Strands SDK when:**
- You want rapid development and iteration
- You're building proof-of-concepts or prototypes
- You want to minimize infrastructure management
- You need flexibility to switch between model providers

**Use Traditional Boto3 when:**
- You need fine-grained control over agent infrastructure
- You have existing Lambda functions to integrate
- You need to manage agents through AWS Console
- You have specific compliance or governance requirements

## Conclusion

In this lab, you've learned how to:

1. Install and set up the Strands Agents SDK
2. Create custom tools using the `@tool` decorator
3. Build an agent that integrates with Amazon Bedrock Knowledge Bases
4. Generate UML diagrams and test code from OpenAPI specifications
5. Invoke agents with simple, Pythonic syntax

The Strands SDK provides a modern, developer-friendly way to build AI agents while maintaining the power and reliability of Amazon Bedrock. For more information, visit:

- [Strands Agents Documentation](https://strandsagents.com/latest/documentation/docs/)
- [Strands SDK GitHub](https://github.com/strands-agents/sdk-python)
- [AWS Bedrock AgentCore](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/)

### Next Steps

- Add more custom tools for your specific use cases
- Deploy your agent to production using Bedrock AgentCore Runtime
- Experiment with different model providers
- Add observability and monitoring using Strands' built-in tracing